## 메가커피 크롤링 & 데이터 전처리
* 메가커피의 현재 점포 수를 정확히 파악하기 위해 메가커피 홈페이지 내 점포 데이터를 크롤링하여 활용하고자 함

# 필요 라이브러리 추가

In [3]:
import requests
from urllib import parse
import pandas as pd
import numpy as np
import seaborn as sns

In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import os

# 주소 카카오API활용 변환 함수 생성

In [5]:
def getGPS_coordinate_for_KAKAO(address):

    headers = {
        'Content-Type': 'application/json; charset=utf-8',
        'Authorization': 'KakaoAK fe0ba99f23346a71189b16b2e0b936f7'
    } # KakaoAK 이후 값들은 발급받은 API키 값(본인 아니면 사용불가)
    address = address.encode("utf-8")

    p = parse.urlencode(
        {
            'query': address
        }
    )

    result = requests.get("https://dapi.kakao.com/v2/local/search/address.json", headers=headers, params=p)
    return result.json()

### 변환 결과값 확인 테스트

In [306]:
getGPS_coordinate_for_KAKAO('서울특별시 마포구 잔다리로6길 39(서교동)')

{'documents': [{'address': {'address_name': '서울 마포구 서교동 369-5',
    'b_code': '1144012000',
    'h_code': '1144066000',
    'main_address_no': '369',
    'mountain_yn': 'N',
    'region_1depth_name': '서울',
    'region_2depth_name': '마포구',
    'region_3depth_h_name': '서교동',
    'region_3depth_name': '서교동',
    'sub_address_no': '5',
    'x': '126.921375976802',
    'y': '37.552438063498'},
   'address_name': '서울 마포구 잔다리로6길 39',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '서울 마포구 잔다리로6길 39',
    'building_name': '',
    'main_building_no': '39',
    'region_1depth_name': '서울',
    'region_2depth_name': '마포구',
    'region_3depth_name': '서교동',
    'road_name': '잔다리로6길',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '126.921375976802',
    'y': '37.552438063498',
    'zone_no': '04039'},
   'x': '126.921375976802',
   'y': '37.552438063498'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

### 복잡한 딕셔너리에서 원하는 값 추출

In [30]:
x = getGPS_coordinate_for_KAKAO('서울특별시 강남구 테헤란로 101 강남역 지하쇼핑센터 e-35호(역삼동)')

### 카카오API key값 확인

In [59]:
x['documents'][0].keys()

dict_keys(['address', 'address_name', 'address_type', 'road_address', 'x', 'y'])

### 주소의 행정코드 확인

In [308]:
x['documents'][0]['address']['h_code']

'1168064000'

### 도로명주소 확인

In [62]:
도로명주소 = x['documents'][0]['road_address']
print(도로명주소)
print()
print(도로명주소['x'])

{'address_name': '서울 강남구 테헤란로 101', 'building_name': '이즈타워', 'main_building_no': '101', 'region_1depth_name': '서울', 'region_2depth_name': '강남구', 'region_3depth_name': '역삼동', 'road_name': '테헤란로', 'sub_building_no': '', 'underground_yn': 'N', 'x': '127.028065727387', 'y': '37.4986232309429', 'zone_no': '06134'}

127.028065727387


# 데이터 전처리 

### 크롤링 후 원하는 형태로 데이터 변환

In [67]:
df = pd.read_html('http://www.megacoffee.me/bbs/board.php?bo_table=store&page=1'.format(e)) # 페이지 형태 맞추어 모두 조회
df2 = df[7].rename(columns=df[7].iloc[0])
df3 = df2.drop(df2.index[0])
df4 = df3.drop(['전화번호','자세히보기'],axis=1)
df4

지역      매장명                                        주소
1  서울      홍대점                  서울특별시 마포구 잔다리로6길 39(서교동)
2  경기   안양1번가점              경기도 안양시 만안구 안양로304번길 13(안양동)
3  경기     성복역점                  경기도 용인시 수지구 수지로 125(성복동)
4  대구   대구동성로점                  대구광역시 중구 중앙대로 395(동성로3가)
5  서울     강남역점  서울특별시 강남구 테헤란로 101 강남역 지하쇼핑센터 e-35호(역삼동)
6  인천     부평역점             인천광역시 부평구 광장로 16, 29-30호(부평동)
7  강원  화천 사창리점                 강원도 화천군 사내면 사내로 40-4(사창리)
8  전북   군산 수송점                     전라북도 군산시 문화로 119(수송동)

### 원하는 형태로 주소 분할 및 위도,경도 추출

In [301]:
군구 = []
세부 = []
위도 = []
경도 = []
for e in df4['주소']:
    if ',' in e :
        address = e.split(',')[0]
        도로명주소 = getGPS_coordinate_for_KAKAO('{}'.format(address))['documents'][0]['address']
        군구.append(도로명주소['region_2depth_name'])
        세부.append(도로명주소['region_3depth_name'])
        위도.append(도로명주소['y'])
        경도.append(도로명주소['x'])
    else :
        도로명주소 = getGPS_coordinate_for_KAKAO('{}'.format(e))['documents'][0]['address'] # 도로명주소 추출
        군구.append(도로명주소['region_2depth_name'])
        세부.append(도로명주소['region_3depth_name'])
        위도.append(도로명주소['y'])
        경도.append(도로명주소['x'])

    군구_np = np.array(군구)
    세부_np = np.array(세부)
    위도_np = np.array(위도)
    경도_np = np.array(경도)

    군구_df = pd.DataFrame(군구_np)
    세부_df = pd.DataFrame(세부_np)
    위도_df = pd.DataFrame(위도_np)
    경도_df = pd.DataFrame(경도_np)

    merge_df = pd.concat([군구_df,세부_df,위도_df,경도_df],axis=1)
    merge_df.index = merge_df.index+1
    newdf = pd.concat([df4,merge_df],axis=1)
newdf.columns = ['지역','매장명','주소','시군구','세부','위도','경도']
newdf

지역      매장명                                           주소       시군구  \
1  부산      대연점                        부산광역시 남구 수영로 262(대연동)        남구   
2  인천     만수역점                       인천광역시 남동구 구월로 358(만수동)       남동구   
3  경기   용인풍덕천점             경기도 용인시 수지구 문인로 57삼익상가동 제1층 104호   용인시 수지구   
4  부산      양정점                     부산광역시 부산진구 중앙대로 898(양정동)      부산진구   
5  충북      진천점                          충청북도 진천군 진천읍 상산로 85       진천군   
6  경기  일산가로수길점            경기도 고양시 일산서구 주엽로 80, E동 113호(대화동)  고양시 일산서구   
7  경기  태전아이파크점                               경기 광주시 태봉로 110       광주시   
8  경기    하남미사점  경기도 하남시 미사강변대로226번안길 25디럭스프라자Ⅰ 1층 111호(망월동)       하남시   

        세부                위도                경도  
1      대연동  35.1353308676929  129.095359915706  
2      만수동  37.4548665524433  126.730338677548  
3     풍덕천동  37.3329514491264  127.096389199643  
4      양정동  35.1703682868454  129.069473251859  
5  진천읍 읍내리  36.8548121663631  127.443587130684  
6      대화동  37.6660625103382  126.756435827812  
7      태전동  37.3836298761114  127.230145922825  
8      망월동  37.5688014599302  127.185252724646

In [131]:
# 위에서 완성된 방식을 전체페이지에 적용. 단, 오류 페이지는 패스

for e in range(1,193):
    # 위에 방법 반복시도
    try: 
        df = pd.read_html('http://www.megacoffee.me/bbs/board.php?bo_table=store&page={}'.format(e)) # 페이지 형태 맞추어 모두 조회
        df2 = df[7].rename(columns=df[7].iloc[0])
        df3 = df2.drop(df2.index[0])
        df4 = df3.drop(['전화번호','자세히보기'],axis=1)
        
        군구 = []
        세부 = []
        위도 = []
        경도 = []      
        
        # print('page{}'.format(e)) -> 작업 진행상황 체크용
        
        for i in df4['주소']:
            if ',' in i :
                address = i.split(',')[0]
                도로명주소 = getGPS_coordinate_for_KAKAO('{}'.format(address))['documents'][0]['road_address']
                군구.append(도로명주소['region_2depth_name'])
                세부.append(도로명주소['region_3depth_name'])
                위도.append(도로명주소['y'])
                경도.append(도로명주소['x'])
            else :
                도로명주소 = getGPS_coordinate_for_KAKAO('{}'.format(i))['documents'][0]['road_address'] # 도로명주소 추출
                군구.append(도로명주소['region_2depth_name'])
                세부.append(도로명주소['region_3depth_name'])
                위도.append(도로명주소['y'])
                경도.append(도로명주소['x'])

            군구_np = np.array(군구)
            세부_np = np.array(세부)
            위도_np = np.array(위도)
            경도_np = np.array(경도)

            군구_df = pd.DataFrame(군구_np)
            세부_df = pd.DataFrame(세부_np)
            위도_df = pd.DataFrame(위도_np)
            경도_df = pd.DataFrame(경도_np)

            merge_df = pd.concat([군구_df,세부_df,위도_df,경도_df],axis=1)
            merge_df.index = merge_df.index+1
            newdf = pd.concat([df4,merge_df],axis=1)
        newdf.columns = ['지역','매장명','주소','시군구','세부','위도','경도']
        newdf.to_csv('./1차프로젝트 데이터/mega_store_list{}.csv'.format(e))
    except:
        pass 

page1
page2
page3
page4
page5
page6
page7
page8
page9
page10
page11
page12
page13
page14
page15
page16
page17
page18
page19
page20
page21
page22
page23
page24
page25
page26
page27
page28
page29
page30
page31
page32
page33
page34
page35
page36
page37
page38
page39
page40
page41
page42
page43
page44
page45
page46
page47
page48
page49
page50
page51
page52
page53
page54
page55
page56
page57
page58
page59
page60
page61
page62
page63
page64
page65
page66
page67
page68
page69
page70
page71
page72
page73
page74
page75
page76
page77
page78
page79
page80
page81
page82
page83
page84
page85
page86
page87
page88
page89
page90
page91
page92
page93
page94
page95
page96
page97
page98
page99
page100
page101
page102
page103
page104
page105
page106
page107
page108
page109
page110
page111
page112
page113
page114
page115
page116
page117
page118
page119
page120
page121
page122
page123
page124
page125
page126
page127
page128
page129
page130
page131
page132
page133
page134
page135
page136
page137
page138
page

# 자동변환 안되는 페이지 문제해결 : 개별 조회 후 변환

### 특이 형태 주소가 있어 자동변환이 안되는 페이지들은 개별 조회 후 변환 

In [292]:
df = pd.read_html('http://www.megacoffee.me/bbs/board.php?bo_table=store&page=25') # 페이지 값 삽입하여 조회
df2 = df[7].rename(columns=df[7].iloc[0])
df3 = df2.drop(df2.index[0])
df4 = df3.drop(['전화번호','자세히보기'],axis=1)
df4

지역      매장명                                           주소
1  부산      대연점                        부산광역시 남구 수영로 262(대연동)
2  인천     만수역점                       인천광역시 남동구 구월로 358(만수동)
3  경기   용인풍덕천점             경기도 용인시 수지구 문인로 57삼익상가동 제1층 104호
4  부산      양정점                     부산광역시 부산진구 중앙대로 898(양정동)
5  충북      진천점                          충청북도 진천군 진천읍 상산로 85
6  경기  일산가로수길점            경기도 고양시 일산서구 주엽로 80, E동 113호(대화동)
7  경기  태전아이파크점         경기도 광주시 태봉로 110태전4지구 C 3블록 106호(태전동)
8  경기    하남미사점  경기도 하남시 미사강변대로226번안길 25디럭스프라자Ⅰ 1층 111호(망월동)

### 개별조회 데이터 지오코딩 가능 값으로 수정

In [299]:
df4.iloc[-2][-1]= '경기 광주시 태봉로 110'
df4

지역      매장명                                           주소
1  부산      대연점                        부산광역시 남구 수영로 262(대연동)
2  인천     만수역점                       인천광역시 남동구 구월로 358(만수동)
3  경기   용인풍덕천점             경기도 용인시 수지구 문인로 57삼익상가동 제1층 104호
4  부산      양정점                     부산광역시 부산진구 중앙대로 898(양정동)
5  충북      진천점                          충청북도 진천군 진천읍 상산로 85
6  경기  일산가로수길점            경기도 고양시 일산서구 주엽로 80, E동 113호(대화동)
7  경기  태전아이파크점                               경기 광주시 태봉로 110
8  경기    하남미사점  경기도 하남시 미사강변대로226번안길 25디럭스프라자Ⅰ 1층 111호(망월동)

In [298]:
# 결과 확인
getGPS_coordinate_for_KAKAO('경기 광주시 태봉로 110')

{'documents': [{'address': {'address_name': '경기 광주시 태전동 735',
    'b_code': '4161011000',
    'h_code': '4161057000',
    'main_address_no': '735',
    'mountain_yn': 'N',
    'region_1depth_name': '경기',
    'region_2depth_name': '광주시',
    'region_3depth_h_name': '광남2동',
    'region_3depth_name': '태전동',
    'sub_address_no': '',
    'x': '127.230145922825',
    'y': '37.3836298761114'},
   'address_name': '경기 광주시 태봉로 110',
   'address_type': 'ROAD_ADDR',
   'road_address': {'address_name': '경기 광주시 태봉로 110',
    'building_name': '태전아이파크',
    'main_building_no': '110',
    'region_1depth_name': '경기',
    'region_2depth_name': '광주시',
    'region_3depth_name': '태전동',
    'road_name': '태봉로',
    'sub_building_no': '',
    'underground_yn': 'N',
    'x': '127.230145922825',
    'y': '37.3836298761114',
    'zone_no': '12787'},
   'x': '127.230145922825',
   'y': '37.3836298761114'}],
 'meta': {'is_end': True, 'pageable_count': 1, 'total_count': 1}}

### 변환가능하게 수정했으면 새롭게 저장

In [300]:
군구 = []
세부 = []
위도 = []
경도 = []
for i in df4['주소']:
    if ',' in i :
        address = i.split(',')[0]
        도로명주소 = getGPS_coordinate_for_KAKAO('{}'.format(address))['documents'][0]['address']
        군구.append(도로명주소['region_2depth_name'])
        세부.append(도로명주소['region_3depth_name'])
        위도.append(도로명주소['y'])
        경도.append(도로명주소['x'])
    else :
        도로명주소 = getGPS_coordinate_for_KAKAO('{}'.format(i))['documents'][0]['address'] # 도로명주소 추출
        군구.append(도로명주소['region_2depth_name'])
        세부.append(도로명주소['region_3depth_name'])
        위도.append(도로명주소['y'])
        경도.append(도로명주소['x'])

        군구_np = np.array(군구)
        세부_np = np.array(세부)
        위도_np = np.array(위도)
        경도_np = np.array(경도)

        군구_df = pd.DataFrame(군구_np)
        세부_df = pd.DataFrame(세부_np)
        위도_df = pd.DataFrame(위도_np)
        경도_df = pd.DataFrame(경도_np)

        merge_df = pd.concat([군구_df,세부_df,위도_df,경도_df],axis=1)
        merge_df.index = merge_df.index+1
        newdf = pd.concat([df4,merge_df],axis=1)
    newdf.columns = ['지역','매장명','주소','시군구','세부','위도','경도']
    newdf.to_csv('./1차프로젝트 데이터/mega_store_list25.csv' # 수정된 페이지 번호입력 후 맞게 저장

In [304]:
# 결과 확인
pd.read_csv('./1차프로젝트 데이터/mega_store_list126.csv')

Unnamed: 0  지역      매장명                          주소      시군구       세부  \
0           1  경기    하남신장점        경기도 하남시 신장로 139(덕풍동)      하남시      덕풍동   
1           2  제주    제주아라점   제주특별자치도 제주시 중앙로 564(아라일동)      제주시     아라일동   
2           3  인천  인천오류왕길점        인천광역시 서구 봉화로 49(오류동)       서구      오류동   
3           4  경북    경북봉화점    경상북도 봉화군 봉화읍 내성로 88(내성리)      봉화군  봉화읍 내성리   
4           5  경기  시흥정왕시장점   경기도 시흥시 정왕천로411번길 25(정왕동)      시흥시      정왕동   
5           6  경기  수원황골마을점    경기도 수원시 영통구 영통로 523(영통동)  수원시 영통구      영통동   
6           7  부산   해운대미포점  부산광역시 해운대구 달맞이길50번길 26(중동)     해운대구       중동   
7           8  전남    목포석현점       전라남도 목포시 옥암로 179(석현동)      목포시      석현동   

          위도          경도  
0  37.538547  127.202769  
1  33.473311  126.545356  
2  37.597692  126.637148  
3  36.890182  128.740667  
4  37.345640  126.749567  
5  37.265529  127.077364  
6  35.160443  129.170716  
7  34.814055  126.421862